In [1]:
import csv
import sys
import numpy as np
import pandas as pd 
import re
from datetime import datetime
import math
import matplotlib
import seaborn as sns
from collections import Counter
%matplotlib inline

### First, remake the Nexus7 dataset to fit SamaungS dataset(colum and rown numbers should be equal).

In [2]:

with open('nexus7.csv', 'r') as f:
    df = pd.read_csv(f, sep=',', low_memory=False, names=['ID','UUID','Language','Hardware_Model','SDK_Version','Manufacture','Screen_Size','Time_Zone','Date_Time','Country_Code','Num_of_CPU_Cores','Location','Location_lat','Location_long','Button','Touch_Pressure','Touch_Size','X_Coordinate','Y_Coordinate','X_Precision','Y_Precision','Action_Type','Action_Timestamp','HR_Timestamp'], skiprows=1)

In [3]:
df=df.drop_duplicates('Action_Timestamp', keep='last')
df=df.reset_index(drop=True)
df.shape

(16510, 24)

In [4]:
event_count = Counter(df["Button"])
event_count

Counter({'.': 998,
         '4': 4,
         '5': 1004,
         'DELETE': 194,
         'DONE': 1000,
         'E': 1,
         'LETTERS': 2051,
         'NUMBERS': 2063,
         'O': 1,
         'R': 405,
         'SHIFT': 1010,
         'SPACE': 2,
         'a': 1008,
         'b': 2,
         'e': 1021,
         'f': 2,
         'i': 1012,
         'k': 2,
         'l': 1014,
         'm': 8,
         'n': 1026,
         'o': 1007,
         'r': 619,
         't': 1008,
         'u': 2,
         'w': 34,
         'z': 12})

In [5]:
ulist=df[['UUID','Action_Timestamp']]
ulist

,UUID,Action_Timestamp
0,test1,499380
1,test1,499450
2,test1,500055
3,test1,500127
4,test1,501088
5,test1,501155
6,test1,502304
7,test1,502396
8,test1,502613
9,test1,502694


In [6]:
def isCorrectPassword(df, i):
    # for idx in range(i, i+32):
    #    print (idx, "\t", df.iloc[idx]["Button"],"\t",df.iloc[idx]["Action_Type"])
    if (df.iloc[i]['Button'] == 'LETTERS' or df.iloc[i]['Button'] == 'NUMBERS') and df.iloc[i]['Action_Type'] == 'Down' and \
                (df.iloc[i+1]['Button'] == 'LETTERS' or df.iloc[i+1]['Button'] == 'NUMBERS') and df.iloc[i+1]['Action_Type'] == 'Up' and \
                df.iloc[i+2]['Button'] == '.' and df.iloc[i+2]['Action_Type'] == 'Down' and \
                df.iloc[i+3]['Button'] == '.' and df.iloc[i+3]['Action_Type'] == 'Up' and \
                (df.iloc[i+4]['Button'] == 'LETTERS' or df.iloc[i+4]['Button'] == 'NUMBERS') and df.iloc[i+4]['Action_Type'] == 'Down' and \
                (df.iloc[i+5]['Button'] == 'LETTERS' or df.iloc[i+5]['Button'] == 'NUMBERS') and df.iloc[i+5]['Action_Type'] == 'Up' and \
                df.iloc[i+6]['Button'] == 't' and df.iloc[i+6]['Action_Type'] == 'Down' and \
                df.iloc[i+7]['Button'] == 't' and df.iloc[i+7]['Action_Type'] == 'Up'and \
                df.iloc[i+8]['Button'] == 'i' and df.iloc[i+8]['Action_Type'] == 'Down' and \
                df.iloc[i+9]['Button'] == 'i' and df.iloc[i+9]['Action_Type'] == 'Up'and \
                df.iloc[i+10]['Button'] == 'e' and df.iloc[i+10]['Action_Type'] == 'Down' and \
                df.iloc[i+11]['Button'] == 'e' and df.iloc[i+11]['Action_Type'] == 'Up' and \
                (df.iloc[i+12]['Button'] == 'LETTERS' or df.iloc[i+12]['Button'] == 'NUMBERS') and df.iloc[i+12]['Action_Type'] == 'Down' and \
                (df.iloc[i+13]['Button'] == 'LETTERS' or df.iloc[i+13]['Button'] == 'NUMBERS') and df.iloc[i+13]['Action_Type'] == 'Up' and \
                df.iloc[i+14]['Button'] == '5' and df.iloc[i+14]['Action_Type'] == 'Down' and \
                df.iloc[i+15]['Button'] == '5' and df.iloc[i+15]['Action_Type'] == 'Up' and \
                (df.iloc[i+16]['Button'] == 'LETTERS' or df.iloc[i+16]['Button'] == 'NUMBERS') and df.iloc[i+16]['Action_Type'] == 'Down' and \
                (df.iloc[i+17]['Button'] == 'LETTERS' or df.iloc[i+17]['Button'] == 'NUMBERS') and df.iloc[i+17]['Action_Type'] == 'Up' and \
                df.iloc[i+18]['Button'] == 'SHIFT' and df.iloc[i+18]['Action_Type'] == 'Down' and \
                df.iloc[i+19]['Button'] == 'SHIFT' and df.iloc[i+19]['Action_Type'] == 'Up' and \
                (df.iloc[i+20]['Button'] == 'R' or df.iloc[i+20]['Button'] == 'r' ) and df.iloc[i+20]['Action_Type'] == 'Down' and \
                (df.iloc[i+21]['Button'] == 'r' or df.iloc[i+21]['Button'] == 'R') and df.iloc[i+21]['Action_Type'] == 'Up' and \
                df.iloc[i+22]['Button'] == 'o' and df.iloc[i+22]['Action_Type'] == 'Down' and \
                df.iloc[i+23]['Button'] == 'o' and df.iloc[i+23]['Action_Type'] == 'Up' and \
                df.iloc[i+24]['Button'] == 'a' and df.iloc[i+24]['Action_Type'] == 'Down' and \
                df.iloc[i+25]['Button'] == 'a' and df.iloc[i+25]['Action_Type'] == 'Up' and \
                df.iloc[i+26]['Button'] == 'n' and df.iloc[i+26]['Action_Type'] == 'Down' and \
                df.iloc[i+27]['Button'] == 'n' and df.iloc[i+27]['Action_Type'] == 'Up' and \
                df.iloc[i+28]['Button'] == 'l' and df.iloc[i+28]['Action_Type'] == 'Down' and \
                df.iloc[i+29]['Button'] == 'l' and df.iloc[i+29]['Action_Type'] == 'Up' and \
                df.iloc[i+30]['Button'] == 'DONE' and df.iloc[i+30]['Action_Type'] == 'Down' and \
                df.iloc[i+31]['Button'] == 'DONE' and df.iloc[i+31]['Action_Type'] == 'Up':
                    # print ("Found a Password that starts at index ",i, " for user ", df.iloc[i]["UUID"])
                    return True
    else:
        return False

In [7]:
ulist = pd.DataFrame(columns = ('UUID','Action_Timestamp'))
df1 = pd.DataFrame()
counter = 0
for i in range(0, len(df)):
        if i+31 < len(df) and isCorrectPassword(df, i):
            df1 = df1.append(df.iloc[i:i+32], ignore_index=True)
            counter = counter +1
            ulist.loc[len(ulist)]=[df.iloc[i]['UUID'],df.iloc[i]['Action_Timestamp']]
users_count = ulist.groupby(['UUID']).Action_Timestamp.nunique()

In [8]:
users_count = users_count.sort_values()
users_count

UUID
test5    71
test4    83
test3    90
test2    97
test1    99
Name: Action_Timestamp, dtype: int64

In [9]:
aList = []
for x in range(0, len(users_count)):
    if (users_count[x]>=50): 
        aList.append(users_count.keys()[x])
aList

['test5', 'test4', 'test3', 'test2', 'test1']

In [10]:
features = pd.DataFrame(columns=("f1","f2","f3","f4","f5","f6","f7","f8","f9","f10","f11","f12","f13","f14","f15","dd2","dd3","dd4","dd5","dd6","dd7","dd9","dd10","dd11","dd12","dd13","dd15","duLN1","du.2","duLN3","dut4","dui5","due6","duLN7","du58","duLN9","duSH10","dur11","duo12","dua13","dun14","dul15","duDO16",\
                                     "UUID"\
                                     ))

for id in aList:

    print('# Adding instances for User ' + str(id))

    idxn = 0

    for i in range(0, len(df)):

        if df.iloc[i]["UUID"] == id and (df.iloc[i]["Button"] == 'LETTERS' or df.iloc[i]["Button"] == 'NUMBERS') and df.iloc[i]["Action_Type"] == 'Down' and idxn < 50:

            features.loc[len(features)] = [\
                                           
                                            #flying time
                                            int(df.iloc[i+2]["Action_Timestamp"]) - int(df.iloc[i+1]["Action_Timestamp"]),\

                                            int(df.iloc[i+4]["Action_Timestamp"]) - int(df.iloc[i+3]["Action_Timestamp"]),\

                                            int(df.iloc[i+6]["Action_Timestamp"]) - int(df.iloc[i+5]["Action_Timestamp"]),\

                                            int(df.iloc[i+8]["Action_Timestamp"]) - int(df.iloc[i+7]["Action_Timestamp"]),\

                                            int(df.iloc[i+10]["Action_Timestamp"]) - int(df.iloc[i+9]["Action_Timestamp"]),\

                                            int(df.iloc[i+12]["Action_Timestamp"]) - int(df.iloc[i+11]["Action_Timestamp"]),\

                                            int(df.iloc[i+14]["Action_Timestamp"]) - int(df.iloc[i+13]["Action_Timestamp"]),\

                                            int(df.iloc[i+16]["Action_Timestamp"]) - int(df.iloc[i+15]["Action_Timestamp"]),\

                                            int(df.iloc[i+18]["Action_Timestamp"]) - int(df.iloc[i+17]["Action_Timestamp"]),\

                                            int(df.iloc[i+20]["Action_Timestamp"]) - int(df.iloc[i+19]["Action_Timestamp"]),\

                                            int(df.iloc[i+22]["Action_Timestamp"]) - int(df.iloc[i+21]["Action_Timestamp"]),\

                                            int(df.iloc[i+24]["Action_Timestamp"]) - int(df.iloc[i+23]["Action_Timestamp"]),\

                                            int(df.iloc[i+26]["Action_Timestamp"]) - int(df.iloc[i+25]["Action_Timestamp"]),\

                                            int(df.iloc[i+28]["Action_Timestamp"]) - int(df.iloc[i+27]["Action_Timestamp"]),\

                                            int(df.iloc[i+30]["Action_Timestamp"]) - int(df.iloc[i+29]["Action_Timestamp"]),\
                                            
                                            #Down-Down
                                            #int(df.iloc[i+2]["Action_Timestamp"]) - int(df.iloc[i]["Action_Timestamp"]),\

                                            int(df.iloc[i+4]["Action_Timestamp"]) - int(df.iloc[i+2]["Action_Timestamp"]),\

                                            int(df.iloc[i+6]["Action_Timestamp"]) - int(df.iloc[i+4]["Action_Timestamp"]),\

                                            int(df.iloc[i+8]["Action_Timestamp"]) - int(df.iloc[i+6]["Action_Timestamp"]),\

                                            int(df.iloc[i+10]["Action_Timestamp"]) - int(df.iloc[i+8]["Action_Timestamp"]),\

                                            int(df.iloc[i+12]["Action_Timestamp"]) - int(df.iloc[i+10]["Action_Timestamp"]),\

                                            int(df.iloc[i+14]["Action_Timestamp"]) - int(df.iloc[i+12]["Action_Timestamp"]),\

                                            #int(df.iloc[i+16]["Action_Timestamp"]) - int(df.iloc[i+14]["Action_Timestamp"]),\

                                            int(df.iloc[i+18]["Action_Timestamp"]) - int(df.iloc[i+16]["Action_Timestamp"]),\

                                            int(df.iloc[i+20]["Action_Timestamp"]) - int(df.iloc[i+18]["Action_Timestamp"]),\

                                            int(df.iloc[i+22]["Action_Timestamp"]) - int(df.iloc[i+20]["Action_Timestamp"]),\

                                            int(df.iloc[i+24]["Action_Timestamp"]) - int(df.iloc[i+22]["Action_Timestamp"]),\

                                            int(df.iloc[i+26]["Action_Timestamp"]) - int(df.iloc[i+24]["Action_Timestamp"]),\

                                            #int(df.iloc[i+28]["Action_Timestamp"]) - int(df.iloc[i+26]["Action_Timestamp"]),\

                                            int(df.iloc[i+30]["Action_Timestamp"]) - int(df.iloc[i+28]["Action_Timestamp"]),\

                                            
                                           
                                            #down-up(hold time)
                                            int(df.iloc[i+1]["Action_Timestamp"]) - int(df.iloc[i]["Action_Timestamp"]),\

                                            int(df.iloc[i+3]["Action_Timestamp"]) - int(df.iloc[i+2]["Action_Timestamp"]),\

                                            int(df.iloc[i+5]["Action_Timestamp"]) - int(df.iloc[i+4]["Action_Timestamp"]),\

                                            int(df.iloc[i+7]["Action_Timestamp"]) - int(df.iloc[i+6]["Action_Timestamp"]),\

                                            int(df.iloc[i+9]["Action_Timestamp"]) - int(df.iloc[i+8]["Action_Timestamp"]),\

                                            int(df.iloc[i+11]["Action_Timestamp"]) - int(df.iloc[i+10]["Action_Timestamp"]),\

                                            int(df.iloc[i+13]["Action_Timestamp"]) - int(df.iloc[i+12]["Action_Timestamp"]),\

                                            int(df.iloc[i+15]["Action_Timestamp"]) - int(df.iloc[i+14]["Action_Timestamp"]),\

                                            int(df.iloc[i+17]["Action_Timestamp"]) - int(df.iloc[i+16]["Action_Timestamp"]),\

                                            int(df.iloc[i+19]["Action_Timestamp"]) - int(df.iloc[i+18]["Action_Timestamp"]),\

                                            int(df.iloc[i+21]["Action_Timestamp"]) - int(df.iloc[i+20]["Action_Timestamp"]),\

                                            int(df.iloc[i+23]["Action_Timestamp"]) - int(df.iloc[i+22]["Action_Timestamp"]),\

                                            int(df.iloc[i+25]["Action_Timestamp"]) - int(df.iloc[i+24]["Action_Timestamp"]),\

                                            int(df.iloc[i+27]["Action_Timestamp"]) - int(df.iloc[i+26]["Action_Timestamp"]),\

                                            int(df.iloc[i+29]["Action_Timestamp"]) - int(df.iloc[i+28]["Action_Timestamp"]),\

                                            int(df.iloc[i+31]["Action_Timestamp"]) - int(df.iloc[i+30]["Action_Timestamp"]),\
                                            


                                            #Touch size for each key entry
                                            #df.iloc[i+1]["Touch_Size"],\

                                            #df.iloc[i+3]["Touch_Size"],\

                                            #df.iloc[i+5]["Touch_Size"],\

                                            #df.iloc[i+7]["Touch_Size"],\

                                            #df.iloc[i+9]["Touch_Size"],\

                                            #df.iloc[i+11]["Touch_Size"],\

                                            #df.iloc[i+13]["Touch_Size"],\

                                            #df.iloc[i+15]["Touch_Size"],\

                                            #df.iloc[i+17]["Touch_Size"],\

                                            #df.iloc[i+19]["Touch_Size"],\

                                            #df.iloc[i+21]["Touch_Size"],\

                                            #df.iloc[i+23]["Touch_Size"],\

                                            #df.iloc[i+25]["Touch_Size"],\

                                            #df.iloc[i+27]["Touch_Size"],\

                                            #df.iloc[i+29]["Touch_Size"],\

                                            #df.iloc[i+31]["Touch_Size"],\
                                            id\

                                            ]

            idxn = idxn +1

            i = i + 32

# Adding instances for User test5
# Adding instances for User test4
# Adding instances for User test3
# Adding instances for User test2
# Adding instances for User test1


In [11]:
features_new = features.drop("f15", axis=1, level=None, inplace=False, errors='raise')
features_new = features.drop("f14", axis=1, level=None, inplace=False, errors='raise')
features_new = features.drop("dun14", axis=1, level=None, inplace=False, errors='raise')
features_new = features.drop("dd15", axis=1, level=None, inplace=False, errors='raise')
features_new = features.drop("dd2", axis=1, level=None, inplace=False, errors='raise')
#features_new = features.drop("dul15", axis=1, level=None, inplace=False, errors='raise')
#features_new = features.drop("duDO16", axis=1, level=None, inplace=False, errors='raise')

### Generate the file and do the test, result=36%

In [12]:
features_new.to_csv('redo_Nexus7.csv')

In [13]:
features_new.shape

(250, 43)

In [14]:
features_impact=features_new.describe()

In [15]:
features_impact

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,duLN7,du58,duLN9,duSH10,dur11,duo12,dua13,dun14,dul15,duDO16
count,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000,250.00000,250.000000,250.000000,250.00000,...,250.000000,250.000000,250.000000,250.000000,250.000000,250.00000,250.000000,250.000000,250.000000,250.000000
mean,382.004000,304.116000,301.600000,331.732000,283.692000,324.948000,331.68400,473.296000,342.804000,447.61600,...,71.044000,71.864000,70.664000,74.100000,72.208000,73.45600,69.300000,73.040000,71.500000,72.028000
std,205.895644,260.018733,261.923147,496.995039,264.131974,501.997553,228.50803,848.905712,276.972686,867.12103,...,16.645913,17.435985,19.540687,15.962116,18.971251,16.49705,16.797937,16.627214,18.339119,17.528268
min,144.000000,24.000000,7.000000,24.000000,7.000000,39.000000,7.00000,53.000000,7.000000,39.00000,...,28.000000,22.000000,10.000000,27.000000,10.000000,22.00000,19.000000,22.000000,10.000000,27.000000
25%,244.250000,157.250000,141.750000,186.500000,97.250000,133.500000,185.25000,212.250000,192.250000,162.00000,...,59.000000,60.000000,58.000000,63.250000,60.000000,62.25000,57.250000,61.000000,58.250000,60.000000
50%,350.500000,242.000000,272.500000,272.000000,245.000000,246.000000,274.50000,344.500000,280.000000,308.00000,...,69.000000,71.000000,69.500000,74.000000,73.000000,73.00000,68.500000,73.000000,73.000000,72.000000
75%,448.750000,369.750000,379.000000,352.500000,389.500000,369.000000,427.50000,525.750000,401.250000,510.75000,...,83.750000,85.000000,84.000000,85.000000,84.000000,85.75000,81.000000,85.000000,83.000000,85.000000
max,1617.000000,2838.000000,2685.000000,7238.000000,2685.000000,7238.000000,1507.00000,12571.000000,2685.000000,12571.00000,...,117.000000,129.000000,136.000000,124.000000,136.000000,124.00000,110.000000,115.000000,129.000000,129.000000


In [16]:
Nexus7_features_mean=features_impact.iloc[[1]]

### To improve the result, we tried to measure the impact between trainning and test data with both mutiply(26%) and plus(22%) saving function, but they both failed.

In [17]:
Nexus7_features_mean

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,duLN7,du58,duLN9,duSH10,dur11,duo12,dua13,dun14,dul15,duDO16
mean,382.004,304.116,301.6,331.732,283.692,324.948,331.684,473.296,342.804,447.616,...,71.044,71.864,70.664,74.1,72.208,73.456,69.3,73.04,71.5,72.028


In [18]:
Nexus7_features_mean.to_csv('Nexus7_features_mean.csv')

In [19]:

with open('SamsungS_features.csv', 'r') as f:
    dfss = pd.read_csv(f, sep=',', low_memory=False)

In [20]:
SamsungS_features_impact=dfss.describe()
SamsungS_features_mean=SamsungS_features_impact.iloc[[1]]
SamsungS_features_mean

,ID,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,duLN7,du58,duLN9,duSH10,dur11,duo12,dua13,dun14,dul15,duDO16
mean,124.5,439.116,350.792,360.08,346.192,340.58,366.136,413.5,1252.62,406.516,...,79.596,75.908,78.524,77.892,80.408,77.86,77.508,75.364,76.02,73.532


In [21]:
SamsungS_features_mean.to_csv('SamsungS_features_mean.csv')

In [22]:
features = pd.DataFrame(columns=("f1","f2","f3","f4","f5","f6","f7","f8","f9","f10","f11","f12","f13","f14","f15","dd2","dd4","dd5","dd6","dd7","dd9","dd10","dd11","dd12","dd13","dd15","duLN1","du.2","duLN3","dut4","dui5","due6","duLN7","du58","duLN9","duSH10","dur11","duo12","dua13","dun14","dul15","duDO16",\
                                     "UUID"\
                                     ))

for id in aList:

    print('# Adding instances for User ' + str(id))

    idxn = 0

    for i in range(0, len(df)):

        if df.iloc[i]["UUID"] == id and (df.iloc[i]["Button"] == 'LETTERS' or df.iloc[i]["Button"] == 'NUMBERS') and df.iloc[i]["Action_Type"] == 'Down' and idxn < 50:

            features.loc[len(features)] = [\
                                           
                                            #flying time
                                            1.15*(int(df.iloc[i+2]["Action_Timestamp"]) - int(df.iloc[i+1]["Action_Timestamp"])),\

                                            1.15*(int(df.iloc[i+4]["Action_Timestamp"]) - int(df.iloc[i+3]["Action_Timestamp"])),\

                                            1.19*(int(df.iloc[i+6]["Action_Timestamp"]) - int(df.iloc[i+5]["Action_Timestamp"])),\

                                            1.04*(int(df.iloc[i+8]["Action_Timestamp"]) - int(df.iloc[i+7]["Action_Timestamp"])),\

                                            1.2*(int(df.iloc[i+10]["Action_Timestamp"]) - int(df.iloc[i+9]["Action_Timestamp"])),\

                                            1.13*(int(df.iloc[i+12]["Action_Timestamp"]) - int(df.iloc[i+11]["Action_Timestamp"])),\

                                            1.25*(int(df.iloc[i+14]["Action_Timestamp"]) - int(df.iloc[i+13]["Action_Timestamp"])),\

                                            2.65*(int(df.iloc[i+16]["Action_Timestamp"]) - int(df.iloc[i+15]["Action_Timestamp"])),\

                                            1.19*(int(df.iloc[i+18]["Action_Timestamp"]) - int(df.iloc[i+17]["Action_Timestamp"])),\

                                            2.7*(int(df.iloc[i+20]["Action_Timestamp"]) - int(df.iloc[i+19]["Action_Timestamp"])),\

                                            1.04*(int(df.iloc[i+22]["Action_Timestamp"]) - int(df.iloc[i+21]["Action_Timestamp"])),\

                                            1.22*(int(df.iloc[i+24]["Action_Timestamp"]) - int(df.iloc[i+23]["Action_Timestamp"])),\

                                            1.17*(int(df.iloc[i+26]["Action_Timestamp"]) - int(df.iloc[i+25]["Action_Timestamp"])),\

                                            1.29*(int(df.iloc[i+28]["Action_Timestamp"]) - int(df.iloc[i+27]["Action_Timestamp"])),\

                                            1.22*(int(df.iloc[i+30]["Action_Timestamp"]) - int(df.iloc[i+29]["Action_Timestamp"])),\
                                            
                                            #Down-Down
                                            #int(df.iloc[i+2]["Action_Timestamp"]) - int(df.iloc[i]["Action_Timestamp"]),\

                                            #int(df.iloc[i+4]["Action_Timestamp"]) - int(df.iloc[i+2]["Action_Timestamp"]),\

                                            1.17*(int(df.iloc[i+6]["Action_Timestamp"]) - int(df.iloc[i+4]["Action_Timestamp"])),\

                                            1.04*(int(df.iloc[i+8]["Action_Timestamp"]) - int(df.iloc[i+6]["Action_Timestamp"])),\

                                            1.17*(int(df.iloc[i+10]["Action_Timestamp"]) - int(df.iloc[i+8]["Action_Timestamp"])),\

                                            1.12*(int(df.iloc[i+12]["Action_Timestamp"]) - int(df.iloc[i+10]["Action_Timestamp"])),\

                                            1.22*(int(df.iloc[i+14]["Action_Timestamp"]) - int(df.iloc[i+12]["Action_Timestamp"])),\

                                            #int(df.iloc[i+16]["Action_Timestamp"]) - int(df.iloc[i+14]["Action_Timestamp"]),\

                                            1.17*(int(df.iloc[i+18]["Action_Timestamp"]) - int(df.iloc[i+16]["Action_Timestamp"])),\

                                            2.47*(int(df.iloc[i+20]["Action_Timestamp"]) - int(df.iloc[i+18]["Action_Timestamp"])),\

                                            1.05*(int(df.iloc[i+22]["Action_Timestamp"]) - int(df.iloc[i+20]["Action_Timestamp"])),\

                                            1.19*(int(df.iloc[i+24]["Action_Timestamp"]) - int(df.iloc[i+22]["Action_Timestamp"])),\

                                            1.16*(int(df.iloc[i+26]["Action_Timestamp"]) - int(df.iloc[i+24]["Action_Timestamp"])),\

                                            #int(df.iloc[i+28]["Action_Timestamp"]) - int(df.iloc[i+26]["Action_Timestamp"]),\

                                            1.2*(int(df.iloc[i+30]["Action_Timestamp"]) - int(df.iloc[i+28]["Action_Timestamp"])),\

                                            
                                           
                                            #down-up(hold time)
                                            1.07*(int(df.iloc[i+1]["Action_Timestamp"]) - int(df.iloc[i]["Action_Timestamp"])),\

                                            1.04*(int(df.iloc[i+3]["Action_Timestamp"]) - int(df.iloc[i+2]["Action_Timestamp"])),\

                                            1.05*(int(df.iloc[i+5]["Action_Timestamp"]) - int(df.iloc[i+4]["Action_Timestamp"])),\

                                            1.05*(int(df.iloc[i+7]["Action_Timestamp"]) - int(df.iloc[i+6]["Action_Timestamp"])),\

                                            1.06*(int(df.iloc[i+9]["Action_Timestamp"]) - int(df.iloc[i+8]["Action_Timestamp"])),\

                                            1.08*(int(df.iloc[i+11]["Action_Timestamp"]) - int(df.iloc[i+10]["Action_Timestamp"])),\

                                            1.12*(int(df.iloc[i+13]["Action_Timestamp"]) - int(df.iloc[i+12]["Action_Timestamp"])),\

                                            1.06*(int(df.iloc[i+15]["Action_Timestamp"]) - int(df.iloc[i+14]["Action_Timestamp"])),\

                                            1.11*(int(df.iloc[i+17]["Action_Timestamp"]) - int(df.iloc[i+16]["Action_Timestamp"])),\

                                            1.05*(int(df.iloc[i+19]["Action_Timestamp"]) - int(df.iloc[i+18]["Action_Timestamp"])),\

                                            1.11*(int(df.iloc[i+21]["Action_Timestamp"]) - int(df.iloc[i+20]["Action_Timestamp"])),\

                                            1.06*(int(df.iloc[i+23]["Action_Timestamp"]) - int(df.iloc[i+22]["Action_Timestamp"])),\

                                            1.12*(int(df.iloc[i+25]["Action_Timestamp"]) - int(df.iloc[i+24]["Action_Timestamp"])),\

                                            1.03*(int(df.iloc[i+27]["Action_Timestamp"]) - int(df.iloc[i+26]["Action_Timestamp"])),\

                                            1.06*(int(df.iloc[i+29]["Action_Timestamp"]) - int(df.iloc[i+28]["Action_Timestamp"])),\

                                            1.02*(int(df.iloc[i+31]["Action_Timestamp"]) - int(df.iloc[i+30]["Action_Timestamp"])),\
                                            


                                            #Touch size for each key entry
                                            #df.iloc[i+1]["Touch_Size"],\

                                            #df.iloc[i+3]["Touch_Size"],\

                                            #df.iloc[i+5]["Touch_Size"],\

                                            #df.iloc[i+7]["Touch_Size"],\

                                            #df.iloc[i+9]["Touch_Size"],\

                                            #df.iloc[i+11]["Touch_Size"],\

                                            #df.iloc[i+13]["Touch_Size"],\

                                            #df.iloc[i+15]["Touch_Size"],\

                                            #df.iloc[i+17]["Touch_Size"],\

                                            #df.iloc[i+19]["Touch_Size"],\

                                            #df.iloc[i+21]["Touch_Size"],\

                                            #df.iloc[i+23]["Touch_Size"],\

                                            #df.iloc[i+25]["Touch_Size"],\

                                            #df.iloc[i+27]["Touch_Size"],\

                                            #df.iloc[i+29]["Touch_Size"],\

                                            #df.iloc[i+31]["Touch_Size"],\
                                            id\

                                            ]

            idxn = idxn +1

            i = i + 32

# Adding instances for User test5
# Adding instances for User test4
# Adding instances for User test3
# Adding instances for User test2
# Adding instances for User test1


In [24]:
features.shape

(250, 43)

In [25]:
features.to_csv('Mutiply_impactSaved_Nexus7.csv')

In [71]:
features = pd.DataFrame(columns=("f1","f2","f3","f4","f5","f6","f7","f8","f9","f10","f11","f12","f13","f14","f15","dd2","dd4","dd5","dd6","dd7","dd9","dd10","dd11","dd12","dd13","dd15","duLN1","du.2","duLN3","dut4","dui5","due6","duLN7","du58","duLN9","duSH10","dur11","duo12","dua13","dun14","dul15","duDO16",\
                                     "UUID"\
                                     ))

for id in aList:

    print('# Adding instances for User ' + str(id))

    idxn = 0

    for i in range(0, len(df)):

        if df.iloc[i]["UUID"] == id and (df.iloc[i]["Button"] == 'LETTERS' or df.iloc[i]["Button"] == 'NUMBERS') and df.iloc[i]["Action_Type"] == 'Down' and idxn < 50:

            features.loc[len(features)] = [\
                                           
                                            #flying time
                                            57+(int(df.iloc[i+2]["Action_Timestamp"]) - int(df.iloc[i+1]["Action_Timestamp"])),\

                                            46+(int(df.iloc[i+4]["Action_Timestamp"]) - int(df.iloc[i+3]["Action_Timestamp"])),\

                                            58+(int(df.iloc[i+6]["Action_Timestamp"]) - int(df.iloc[i+5]["Action_Timestamp"])),\

                                            14+(int(df.iloc[i+8]["Action_Timestamp"]) - int(df.iloc[i+7]["Action_Timestamp"])),\

                                            56+(int(df.iloc[i+10]["Action_Timestamp"]) - int(df.iloc[i+9]["Action_Timestamp"])),\

                                            41+(int(df.iloc[i+12]["Action_Timestamp"]) - int(df.iloc[i+11]["Action_Timestamp"])),\

                                            81+(int(df.iloc[i+14]["Action_Timestamp"]) - int(df.iloc[i+13]["Action_Timestamp"])),\

                                            779+(int(df.iloc[i+16]["Action_Timestamp"]) - int(df.iloc[i+15]["Action_Timestamp"])),\

                                            63+(int(df.iloc[i+18]["Action_Timestamp"]) - int(df.iloc[i+17]["Action_Timestamp"])),\

                                            761+(int(df.iloc[i+20]["Action_Timestamp"]) - int(df.iloc[i+19]["Action_Timestamp"])),\

                                            14+(int(df.iloc[i+22]["Action_Timestamp"]) - int(df.iloc[i+21]["Action_Timestamp"])),\

                                            55+(int(df.iloc[i+24]["Action_Timestamp"]) - int(df.iloc[i+23]["Action_Timestamp"])),\

                                            54+(int(df.iloc[i+26]["Action_Timestamp"]) - int(df.iloc[i+25]["Action_Timestamp"])),\

                                            137+(int(df.iloc[i+28]["Action_Timestamp"]) - int(df.iloc[i+27]["Action_Timestamp"])),\

                                            74+(int(df.iloc[i+30]["Action_Timestamp"]) - int(df.iloc[i+29]["Action_Timestamp"])),\
                                            
                                            #Down-Down
                                            #int(df.iloc[i+2]["Action_Timestamp"]) - int(df.iloc[i]["Action_Timestamp"]),\

                                            #int(df.iloc[i+4]["Action_Timestamp"]) - int(df.iloc[i+2]["Action_Timestamp"]),\

                                            61+(int(df.iloc[i+6]["Action_Timestamp"]) - int(df.iloc[i+4]["Action_Timestamp"])),\

                                            17+(int(df.iloc[i+8]["Action_Timestamp"]) - int(df.iloc[i+6]["Action_Timestamp"])),\

                                            61+(int(df.iloc[i+10]["Action_Timestamp"]) - int(df.iloc[i+8]["Action_Timestamp"])),\

                                            46+(int(df.iloc[i+12]["Action_Timestamp"]) - int(df.iloc[i+10]["Action_Timestamp"])),\

                                            90+(int(df.iloc[i+14]["Action_Timestamp"]) - int(df.iloc[i+12]["Action_Timestamp"])),\

                                            #int(df.iloc[i+16]["Action_Timestamp"]) - int(df.iloc[i+14]["Action_Timestamp"]),\

                                            71+(int(df.iloc[i+18]["Action_Timestamp"]) - int(df.iloc[i+16]["Action_Timestamp"])),\

                                            765+(int(df.iloc[i+20]["Action_Timestamp"]) - int(df.iloc[i+18]["Action_Timestamp"])),\

                                            22+(int(df.iloc[i+22]["Action_Timestamp"]) - int(df.iloc[i+20]["Action_Timestamp"])),\

                                            60+(int(df.iloc[i+24]["Action_Timestamp"]) - int(df.iloc[i+22]["Action_Timestamp"])),\

                                            62+(int(df.iloc[i+26]["Action_Timestamp"]) - int(df.iloc[i+24]["Action_Timestamp"])),\

                                            #int(df.iloc[i+28]["Action_Timestamp"]) - int(df.iloc[i+26]["Action_Timestamp"]),\

                                            79+(int(df.iloc[i+30]["Action_Timestamp"]) - int(df.iloc[i+28]["Action_Timestamp"])),\

                                            
                                           
                                            #down-up(hold time)
                                            5+(int(df.iloc[i+1]["Action_Timestamp"]) - int(df.iloc[i]["Action_Timestamp"])),\

                                            2+(int(df.iloc[i+3]["Action_Timestamp"]) - int(df.iloc[i+2]["Action_Timestamp"])),\

                                            3+(int(df.iloc[i+5]["Action_Timestamp"]) - int(df.iloc[i+4]["Action_Timestamp"])),\

                                            3+(int(df.iloc[i+7]["Action_Timestamp"]) - int(df.iloc[i+6]["Action_Timestamp"])),\

                                            4+(int(df.iloc[i+9]["Action_Timestamp"]) - int(df.iloc[i+8]["Action_Timestamp"])),\

                                            5+(int(df.iloc[i+11]["Action_Timestamp"]) - int(df.iloc[i+10]["Action_Timestamp"])),\

                                            8+(int(df.iloc[i+13]["Action_Timestamp"]) - int(df.iloc[i+12]["Action_Timestamp"])),\

                                            4+(int(df.iloc[i+15]["Action_Timestamp"]) - int(df.iloc[i+14]["Action_Timestamp"])),\

                                            7+(int(df.iloc[i+17]["Action_Timestamp"]) - int(df.iloc[i+16]["Action_Timestamp"])),\

                                            3+(int(df.iloc[i+19]["Action_Timestamp"]) - int(df.iloc[i+18]["Action_Timestamp"])),\

                                            8+(int(df.iloc[i+21]["Action_Timestamp"]) - int(df.iloc[i+20]["Action_Timestamp"])),\

                                            4+(int(df.iloc[i+23]["Action_Timestamp"]) - int(df.iloc[i+22]["Action_Timestamp"])),\

                                            8+(int(df.iloc[i+25]["Action_Timestamp"]) - int(df.iloc[i+24]["Action_Timestamp"])),\

                                            2+(int(df.iloc[i+27]["Action_Timestamp"]) - int(df.iloc[i+26]["Action_Timestamp"])),\

                                            4+(int(df.iloc[i+29]["Action_Timestamp"]) - int(df.iloc[i+28]["Action_Timestamp"])),\

                                            1+(int(df.iloc[i+31]["Action_Timestamp"]) - int(df.iloc[i+30]["Action_Timestamp"])),\
                                            


                                            #Touch size for each key entry
                                            #df.iloc[i+1]["Touch_Size"],\

                                            #df.iloc[i+3]["Touch_Size"],\

                                            #df.iloc[i+5]["Touch_Size"],\

                                            #df.iloc[i+7]["Touch_Size"],\

                                            #df.iloc[i+9]["Touch_Size"],\

                                            #df.iloc[i+11]["Touch_Size"],\

                                            #df.iloc[i+13]["Touch_Size"],\

                                            #df.iloc[i+15]["Touch_Size"],\

                                            #df.iloc[i+17]["Touch_Size"],\

                                            #df.iloc[i+19]["Touch_Size"],\

                                            #df.iloc[i+21]["Touch_Size"],\

                                            #df.iloc[i+23]["Touch_Size"],\

                                            #df.iloc[i+25]["Touch_Size"],\

                                            #df.iloc[i+27]["Touch_Size"],\

                                            #df.iloc[i+29]["Touch_Size"],\

                                            #df.iloc[i+31]["Touch_Size"],\
                                            id\

                                            ]

            idxn = idxn +1

            i = i + 32

# Adding instances for User test5
# Adding instances for User test4
# Adding instances for User test3
# Adding instances for User test2
# Adding instances for User test1


In [73]:
features.to_csv('Plus_impactSaved_Nexus7.csv')